### 1 - Imports

In [172]:
import os
from dotenv import load_dotenv
from firecrawl import FirecrawlApp
import tiktoken
import difflib
import pandas as pd

In [105]:
load_dotenv()

True

### 2 - OpenAI Azure model deployement

In [106]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv('Azure_OpenAI_OB_Endpoint'), 
    openai_api_version="2024-02-15-preview",
    model_name="gpt-4o",
    openai_api_key=os.getenv('Azure_OpenAI_OB_Key'), 
    openai_api_type="azure",
    temperature=0,
    deployment_name="gpt-4o-deploy")
    #streaming=True

In [142]:
prompt = "Fais une blague sur les actuaires"

for chunk in llm.stream(prompt):
    print(chunk.content, end="", flush=True)

Pourquoi les actuaires ne racontent-ils jamais de blagues ?

Parce que les probabilités que quelqu'un rie sont trop faibles !

### 3 - Data Loader Langchain KO

In [108]:
from langchain_community.document_loaders import UnstructuredURLLoader

url_OB = ["http://publications.europa.eu/resource/celex/32015L0849"]

In [109]:
loader = UnstructuredURLLoader(urls=url_OB)

data = loader.load()


libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
Error fetching or processing http://publications.europa.eu/resource/celex/32015L0849, exception: Partitioning is not supported for the FileType.UNK file type.


In [110]:
if data:
    print(data[0])  # Afficher le premier élément extrait
else:
    print("Aucun contenu récupéré.")

Aucun contenu récupéré.


### 3 -  Data Ingestion (firecrawl)

In [111]:
app = FirecrawlApp(api_key="fc-3be59290354d478ca821956a0aa86e41")

In [112]:
url_auto= "http://publications.europa.eu/resource/celex/32015L0849"
# url_fr ='http://publications.europa.eu/resource/cellar/0bff31ef-0b49-11e5-8817-01aa75ed71a1.0009.03/DOC_1'

In [113]:
scrape_result = app.scrape_url(
    url_auto,
    params={
        'formats': ['markdown', 'html'],
        'location': {
            'country': 'FR'
        }
    }
)
print(scrape_result)

{'markdown': "|     |     |     |     |\n| --- | --- | --- | --- |\n| 5.6.2015 | FR | Journal officiel de l'Union européenne | L 141/73 |\n\n* * *\n\nDIRECTIVE (UE) 2015/849 DU PARLEMENT EUROPÉEN ET DU CONSEIL\n\ndu 20 mai 2015\n\nrelative à la prévention de l'utilisation du système financier aux fins du blanchiment de capitaux ou du financement du terrorisme, modifiant le règlement (UE) no\xa0648/2012 du Parlement européen et du Conseil et abrogeant la directive 2005/60/CE du Parlement européen et du Conseil et la directive 2006/70/CE de la Commission\n\n(Texte présentant de l'intérêt pour l'EEE)\n\nLE PARLEMENT EUROPÉEN ET LE CONSEIL DE L'UNION EUROPÉENNE,\n\nvu le traité sur le fonctionnement de l'Union européenne, et notamment son article\xa0114,\n\nvu la proposition de la Commission européenne,\n\naprès transmission du projet d'acte législatif aux parlements nationaux,\n\nvu l'avis de la Banque centrale européenne [(1)](https://publications.europa.eu/resource/cellar/0bff31ef-0b49-

In [114]:
scrape_result.keys()

dict_keys(['markdown', 'metadata', 'html'])

In [144]:
directive_UE =  scrape_result["markdown"] 

### 4 - Long Context Question Answering (Option 1)

Pour le versionning de prompts dans Python, plusieurs outils possibles :
- PromptLayer
- LangSmith , 
- Agenta, 
- Mirascope et Lilypad
- prompt flow

In [162]:
fr_old_version= "Les entreprises établies en France et qui font partie d'un groupe financier ou d'un groupe comprenant au moins une société de financement ou, pour l'application du 2° du présent article, d'un groupe au sens de l'article L. 356-1 du code des assurances, ou d'un groupe mixte ou d'un conglomérat financier auquel appartiennent des entités réglementées au sens de l'article L. 517-2 ayant leur siège social dans un Etat membre de l'Union européenne ou Etat partie à l'accord sur l'Espace économique européen ou dans un Etat où sont applicables les accords prévus aux articles L. 632-7 , L. 632-13 et L. 632-16 du présent code sont tenues, nonobstant toutes dispositions contraires, de transmettre à des entreprises du même groupe ayant leur siège social dans l'un de ces Etats : 1° Les renseignements relatifs à leur situation financière nécessaires à l'organisation de la surveillance sur base consolidée et de la surveillance complémentaire de ces entités réglementées ou sociétés de financement ; 2° Les informations nécessaires à l'organisation de la lutte contre le blanchiment des capitaux et contre le financement du terrorisme ; 3° Les informations nécessaires à l'organisation de la détection des abus de marché mentionnés à l'article 16 du règlement (UE) n° 596/2014 du Parlement européen et du Conseil du 16 avril 2014 sur les abus de marché (règlement relatif aux abus de marché) et abrogeant la directive 2003/6/CE du Parlement européen et du Conseil et les directives 2003/124/CE, 2003/125/CE et 2004/72/CE de la Commission ; 4° Les informations nécessaires à la gestion des conflits d'intérêts au sens du 3° du I et 3° du II de l'article L. 533-10 . Ces dernières informations ne peuvent être communiquées à des personnes extérieures au groupe, à l'exception des autorités compétentes des Etats visés au premier alinéa. Cette exception ne s'étend pas aux autorités des Etats ou territoires dont la législation est reconnue insuffisante ou dont les pratiques sont considérées comme faisant obstacle à la lutte contre le blanchiment des capitaux ou le financement du terrorisme par l'instance internationale de concertation et de coordination en matière de lutte contre le blanchiment d'argent dont la liste est mise à jour par arrêté du ministre chargé de l'économie. Les personnes recevant ces informations sont tenues au secret professionnel dans les conditions et sous les peines mentionnées à l'article L. 511-33 , pour tous renseignements ou documents qu'elles seraient ainsi amenées à recevoir ou à détenir. Les dispositions du présent article ne font pas obstacle à l'application de la loi n° 78-17 du 6 janvier 1978 relative à l'informatique, aux fichiers et aux libertés."

In [163]:
fr_new_version= "Les entreprises établies en France et qui font partie d'un groupe financier ou d'un groupe comprenant au moins une société de financement ou, pour l'application du 2° du présent article, d'un groupe au sens de l'article L. 356-1 du code des assurances, d'un groupe au sens du III de l'article L. 511-20 comprenant au moins une entité mentionnée au 1° bis ou 1° ter de l'article L. 561-2, ou d'un groupe mixte ou d'un conglomérat financier auquel appartiennent des entités réglementées au sens de l'article L. 517-2 ayant leur siège social dans un Etat membre de l'Union européenne ou Etat partie à l'accord sur l'Espace économique européen ou dans un Etat où sont applicables les accords prévus aux articles L. 632-7 , L. 632-13 et L. 632-16 du présent code sont tenues, nonobstant toutes dispositions contraires, de transmettre à des entreprises du même groupe ayant leur siège social dans l'un de ces Etats : 1° Les renseignements relatifs à leur situation financière nécessaires à l'organisation de la surveillance sur base consolidée et de la surveillance complémentaire de ces entités réglementées ou sociétés de financement ; 2° Les informations nécessaires à l'organisation de la lutte contre le blanchiment des capitaux et contre le financement du terrorisme ; 3° Les informations nécessaires à l'organisation de la détection des abus de marché mentionnés à l'article 16 du règlement (UE) n° 596/2014 du Parlement européen et du Conseil du 16 avril 2014 sur les abus de marché (règlement relatif aux abus de marché) et abrogeant la directive 2003/6/CE du Parlement européen et du Conseil et les directives 2003/124/CE, 2003/125/CE et 2004/72/CE de la Commission ; 4° Les informations nécessaires à la gestion des conflits d'intérêts au sens du 3° du I et 3° du II de l'article L. 533-10 . Ces dernières informations ne peuvent être communiquées à des personnes extérieures au groupe, à l'exception des autorités compétentes des Etats visés au premier alinéa. Cette exception ne s'étend pas aux autorités des Etats ou territoires dont la législation est reconnue insuffisante ou dont les pratiques sont considérées comme faisant obstacle à la lutte contre le blanchiment des capitaux ou le financement du terrorisme par l'instance internationale de concertation et de coordination en matière de lutte contre le blanchiment d'argent dont la liste est mise à jour par arrêté du ministre chargé de l'économie. Les personnes recevant ces informations sont tenues au secret professionnel dans les conditions et sous les peines mentionnées à l'article L. 511-33 , pour tous renseignements ou documents qu'elles seraient ainsi amenées à recevoir ou à détenir. Les dispositions du présent article ne font pas obstacle à l'application de la loi n° 78-17 du 6 janvier 1978 relative à l'informatique, aux fichiers et aux libertés."

In [ ]:
# LLM produit : analyse du changement avant et après et recherche de l article UE a l origine du changement.
prompt_LCQA_3p = f""" 
        Tu es un juriste expert avec au moins 10 ans d expérience.
        Tu as une double expertise a la fois dans la réglementation francaise et dans les textes européens.
        Tu as travaillé a plusieurs reprises sur la transposition des textes européens dans la réglementation francaise.
        
        Ta tâche :
        -   Analyser les changements réglementaires dans la loi francaise avec pour objectif de trouver la ou les parties
        de la directive UE a l origine de ce(s) changement(s).
        
        Régles : 
        - Tu te baseras uniquement sur les informations fournies pour faire ton analyse.
        - Tu feras ton analyse de la facon la plus conconcieuse et précise 
        - Tu ne pas pas reciter le contenu de la loi francaise avant et après modification
        
        Voici le text européen {directive_UE}
        Voici le texte de loi francais dans sa version avant l'entree en vigueur du text européen: {fr_old_version}
        Voici le texte de loi francais dans sa version après l'entree en vigueur du text européen: {fr_new_version}

"""

In [165]:
for chunk in llm.stream(prompt_LCQA_3p):
    print(chunk.content, end="", flush=True)

L'analyse des changements réglementaires dans la loi française et leur lien avec la directive européenne 2015/849 du Parlement européen et du Conseil du 20 mai 2015 révèle plusieurs points de convergence. Voici les principaux éléments de la directive européenne qui ont influencé les modifications de la loi française :

1. **Extension du champ d'application** :
   - La directive européenne 2015/849, dans son article 2, étend le champ d'application aux entités assujetties, y compris les établissements de crédit, les établissements financiers, et les autres entités impliquées dans la lutte contre le blanchiment de capitaux et le financement du terrorisme. La modification de la loi française inclut désormais les entités mentionnées au 1° bis ou 1° ter de l'article L. 561-2, ce qui correspond à l'élargissement du champ d'application prévu par la directive.

2. **Transmission des informations** :
   - L'article 45 de la directive européenne exige des entités assujetties qu'elles mettent en œ

In [173]:
df1 = pd.DataFrame({"Ancienne": [fr_old_version] , 
                   "Nouvelle": [fr_new_version]})

In [170]:
def comparer_phrases(phrase1, phrase2):
    mots1 = phrase1.split()
    mots2 = phrase2.split()
    sequence = difflib.SequenceMatcher(None, mots1, mots2)
    differences = []
    for tag, i1, i2, j1, j2 in sequence.get_opcodes():
        if tag == 'replace':
            differences.append(f"Remplacer '{' '.join(mots1[i1:i2])}' par '{' '.join(mots2[j1:j2])}'")
        elif tag == 'delete':
            differences.append(f"Supprimer '{' '.join(mots1[i1:i2])}'")
        elif tag == 'insert':
            differences.append(f"Insérer '{' '.join(mots2[j1:j2])}'")
    return differences

In [175]:
changement_derter= comparer_phrases(fr_old_version, fr_new_version)
print(changement_derter)

["Insérer 'd'un groupe au sens du III de l'article L. 511-20 comprenant au moins une entité mentionnée au 1° bis ou 1° ter de l'article L. 561-2,'"]


In [176]:
prompt_LCQA_4p = f""" 
        Tu es un juriste expert avec au moins 10 ans d expérience.
        Tu as une double expertise a la fois dans la réglementation francaise et dans les textes européens.
        Tu as travaillé a plusieurs reprises sur la transposition des textes européens dans la réglementation francaise.
        
        
        Ta tâche :
        -   Analyser les changements réglementaires dans la loi francaise et de trouver la ou les parties
        de la directive UE a l origine de ce changement réglementaire.
        
        Régles : 
        - Tu te baseras uniquement sur les informations fournies pour faire ton analyse.
        - Tu feras ton analyse de la facon la plus conconcieuse et précise 
        
        Voici le text européen {directive_UE}
        Voici le texte de loi francais dans sa version avant l'entree en vigueur du text européen: {fr_old_version}
        Voici le texte de loi francais dans sa version après l'entree en vigueur du text européen: {fr_new_version}
        Voici une indication des changements entre les deux version du texte francais sur la quelle tu t appuieras pour
        trouver la partie du texte europeen a l origine de ce changement : {changement_derter}

"""

In [177]:
for chunk in llm.stream(prompt_LCQA_4p):
    print(chunk.content, end="", flush=True)

Pour analyser les changements réglementaires dans la loi française et identifier la partie de la directive européenne à l'origine de ce changement, nous devons examiner les modifications apportées au texte français et les comparer avec les dispositions de la directive européenne.

### Changements dans le texte français

La modification apportée au texte français est l'insertion de la phrase suivante :
**"d'un groupe au sens du III de l'article L. 511-20 comprenant au moins une entité mentionnée au 1° bis ou 1° ter de l'article L. 561-2,"**

### Analyse de la directive européenne

La directive européenne 2015/849 du Parlement européen et du Conseil du 20 mai 2015 relative à la prévention de l'utilisation du système financier aux fins du blanchiment de capitaux ou du financement du terrorisme contient plusieurs articles et considérants qui traitent de la lutte contre le blanchiment de capitaux et le financement du terrorisme. 

### Correspondance avec la directive européenne

La partie d

### 5 - Retrieval Augmented Generation   

#### 5.1 - Chunking  

#### 5.1.1 - Counting doc tokens

In [ ]:
def count_tokens(text: str, model: str = "gpt-4o") -> int:
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)


In [ ]:
text = scrape_result["markdown"]
type(text)

str

In [ ]:
count_tokens(text)

50960

#### 4.2 - Extract structure with LLM

- PromptLayer
	•	Interface visuelle pour éditer et déployer des prompts sans code
	•	Contrôle de version à l’échelle de l’entreprise avec historique détaillé
	•	Tableau de bord analytique pour suivre les performances
	•	Tests A/B intégrés pour évaluer les prompts
	•	Support multi-modèles avec système de template unifié


- Agenta
	•	Simplifie les tests de prompts avec contrôle de version
	•	Permet des comparaisons côte à côte entre différents LLM
	•	Plateforme open source pour le développement d’applications LLM

- LangSmith
	•	Hub centralisé (LangChain Hub) pour archiver et versionner les prompts
	•	Intégration avec LangChain pour gérer les chaînes et agents intelligents
	•	Commandes pour télécharger des prompts spécifiques avec leur hash de version

- Mirascope et Lilypad
	•	Toolkit Python léger pour construire des applications LLM de qualité production
	•	Système de gestion et versionnage de prompts intégré (Lilypad)
	•	Permet l’utilisation locale avec vos prompts LLM
	•	Regroupe tous les éléments affectant la qualité d’un appel LLM en une unité cohésive

- Pezzo
	•	Plateforme LLMOps open source pour l’ingénierie de prompts
	•	Interface graphique pour créer, tester et versionner les prompts
	•	Utilise des hashes SHA comme numéros de version

In [143]:
prompt_v1 = f"""
        You are an expert data scientist and data engineer with an expertise in law .
        That means that you have a deep knowlage of law.
        
        To do: 
          - Extract the structure of this document to prepare to make the best and most reliable embedding.
          
        Rules: 
          - Be as acurate as possible.
          - Do not ommit information.
          - Do not omit the structure under "Article"
          - Do not translate the text : 
          {text} 
          """

for chunk in llm.stream(prompt):
    print(chunk.content, end="", flush=True)

Pourquoi les actuaires ne racontent-ils jamais de blagues ?

Parce que les probabilités que quelqu'un rie sont trop faibles !